In [1]:
import requests
import matplotlib.pyplot as plt

# response = requests.get("https://blockstream.info/api/blocks")

# how_many = 20

# def get_blocks(how_many):
#     response = requests.get("https://blockstream.info/api/blocks")
#     blocks = response.json()
#     return blocks[:how_many]

#еще не доделал, не ебу как это работает. Чат написал как обойти апи 
def get_blocks_n(n):
    blocks = []
    last_block_hash = None

    while len(blocks) < n:
        if last_block_hash:
            url = f"https://blockstream.info/api/blocks/{last_block_hash}"
        else:
            url = "https://blockstream.info/api/blocks"
        
        response = requests.get(url)
        batch = response.json()

        if not batch:
            break  # блоки закончились

        blocks.extend(batch)

        # Обновляем хэш последнего блока из текущей партии
        last_block_hash = batch[-1]['id']

    return blocks[:n]  # вернём ровно n блоков

# Пример использования
how_many = 100
blocks = get_blocks_n(how_many)

In [2]:
def get_transactions(block_hash):
    url = f"https://blockstream.info/api/block/{block_hash}/txs"
    response = requests.get(url)
    transactions = response.json()
    return transactions

In [4]:
def analyze_blocks(how_many):
    blocks = get_blocks_n(how_many)
    block_heights = []
    total_sums_btc = []  
    
    for block in blocks:
        print(f"Block: {block['height']} (ID: {block['id']})")
        txs = get_transactions(block['id'])
        print(f"How many transactions in Block: {len(txs)}")
        print(f"Size of Block: {block['size']} bytes")

        total_sum = 0
        for tx in txs:
            for summ in tx['vout']:
                total_sum += summ['value']

        total_btc = total_sum / 100_000_000
        print(f"Summ: {total_btc:.8f} BTC")
        print("-" * 40)

        block_heights.append(block['height'])
        total_sums_btc.append(total_btc)

    

analyze_blocks(how_many)

Block: 897411 (ID: 0000000000000000000006eed420109176b48046f7dfea28c9342a2defdc038d)
How many transactions in Block: 25
Size of Block: 1560787 bytes
Summ: 30.30586131 BTC
----------------------------------------
Block: 897410 (ID: 000000000000000000001561035eb56e8137b91a28a14fa505a2d428d3238d5e)
How many transactions in Block: 25
Size of Block: 1332750 bytes
Summ: 75.49614845 BTC
----------------------------------------
Block: 897409 (ID: 00000000000000000001a48c9d895098ca23a3ef14c4f2d0fbb2694e761a99c2)
How many transactions in Block: 25
Size of Block: 1593918 bytes
Summ: 7.35066511 BTC
----------------------------------------
Block: 897408 (ID: 0000000000000000000126f6e4dbe6fc6443e98f133af98847aa491ec9e27cac)
How many transactions in Block: 25
Size of Block: 1672570 bytes
Summ: 8.60768225 BTC
----------------------------------------
Block: 897407 (ID: 00000000000000000000f279df67b118dc692e51c74081a0d660c3d5bb5ddbc0)
How many transactions in Block: 25
Size of Block: 1606659 bytes
Summ: